In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mediapipe as mp
import cv2
import os

In [2]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(min_detection_confidence=0.6,
                    min_tracking_confidence=0.6,
                    static_image_mode=False,
                    smooth_landmarks=True,
                    model_complexity=2)

In [3]:
desired_width = 480
desired_height = 360

In [4]:
def calculate_angle(row):
    import math
    row = abs(row)

    # Accessing columns by their index positions (0 to 5) from the row
    radians = math.atan2(row[5] - row[3], row[4] - row[2]) - math.atan2(row[1] - row[3], row[0] - row[2])
    angle = math.degrees(radians)
    if angle < 0:
        angle += 360
    return angle

In [5]:
def processVideo(VideoFile, positive=True):
    cap = cv2.VideoCapture(VideoFile)
    
    data_dict = {}
    
    for landmark in mp_pose.PoseLandmark:
        x_list_name = f"{landmark.name}_Point_x"
        y_list_name = f"{landmark.name}_Point_y"
        z_list_name = f"{landmark.name}_Point_z"
        visibility_list_name = f"{landmark.name}_visibility"
    
        data_dict[x_list_name] = []
        data_dict[y_list_name] = []
        data_dict[z_list_name] = []
        data_dict[visibility_list_name] = []
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break

        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        if results.pose_landmarks:

            for landmark in mp_pose.PoseLandmark:
                x_list_name = f"{landmark.name}_Point_x"
                y_list_name = f"{landmark.name}_Point_y"
                z_list_name = f"{landmark.name}_Point_z"
                visibility_list_name = f"{landmark.name}_visibility"

                data_dict[x_list_name].append(results.pose_landmarks.landmark[landmark].x)
                data_dict[y_list_name].append(results.pose_landmarks.landmark[landmark].y)
                data_dict[z_list_name].append(results.pose_landmarks.landmark[landmark].z)
                data_dict[visibility_list_name].append(results.pose_landmarks.landmark[landmark].visibility)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    df = pd.DataFrame(data_dict)
    Angles_df = pd.DataFrame()

    Angle_At_Right_Elbow = df[[
        'RIGHT_WRIST_Point_x', 'RIGHT_WRIST_Point_y',
        'RIGHT_ELBOW_Point_x', 'RIGHT_ELBOW_Point_y',
        'RIGHT_SHOULDER_Point_x', 'RIGHT_SHOULDER_Point_y'
    ]]

    Angles_df['Angle_At_Right_Elbow'] = Angle_At_Right_Elbow.apply(calculate_angle, axis=1)

    Angle_At_Left_Elbow = df[[
        'LEFT_WRIST_Point_x', 'LEFT_WRIST_Point_y',
        'LEFT_ELBOW_Point_x', 'LEFT_ELBOW_Point_y',
        'LEFT_SHOULDER_Point_x', 'LEFT_SHOULDER_Point_y'
    ]]
    Angles_df['Angle_At_Left_Elbow'] = Angle_At_Left_Elbow.apply(calculate_angle, axis=1)

    # Calculate angle at right shoulder
    Angle_At_Right_Shoulder = df[[
        'RIGHT_ELBOW_Point_x', 'RIGHT_ELBOW_Point_y',
        'RIGHT_SHOULDER_Point_x', 'RIGHT_SHOULDER_Point_y',
        'RIGHT_HIP_Point_x', 'RIGHT_HIP_Point_y'
    ]]
    Angles_df['Angle_At_Right_Shoulder'] = Angle_At_Right_Shoulder.apply(calculate_angle, axis=1)

    # Calculate angle at left shoulder
    Angle_At_Left_Shoulder = df[[
        'LEFT_ELBOW_Point_x', 'LEFT_ELBOW_Point_y',
        'LEFT_SHOULDER_Point_x', 'LEFT_SHOULDER_Point_y',
        'LEFT_HIP_Point_x', 'LEFT_HIP_Point_y'
    ]]
    Angles_df['Angle_At_Left_Shoulder'] = Angle_At_Left_Shoulder.apply(calculate_angle, axis=1)

    # Calculate angle at right knee (example)
    Angle_At_Right_Knee = df[[
        'RIGHT_HIP_Point_x', 'RIGHT_HIP_Point_y',
        'RIGHT_KNEE_Point_x', 'RIGHT_KNEE_Point_y',
        'RIGHT_ANKLE_Point_x', 'RIGHT_ANKLE_Point_y'
    ]]
    Angles_df['Angle_At_Right_Knee'] = Angle_At_Right_Knee.apply(calculate_angle, axis=1)

    # Calculate angle at left knee (example)
    Angle_At_Left_Knee = df[[
        'LEFT_HIP_Point_x', 'LEFT_HIP_Point_y',
        'LEFT_KNEE_Point_x', 'LEFT_KNEE_Point_y',
        'LEFT_ANKLE_Point_x', 'LEFT_ANKLE_Point_y'
    ]]
    Angles_df['Angle_At_Left_Knee'] = Angle_At_Left_Knee.apply(calculate_angle, axis=1)

    # Calculate angle at right hip
    Angle_At_Right_Hip = df[[
        'RIGHT_SHOULDER_Point_x', 'RIGHT_SHOULDER_Point_y',
        'RIGHT_HIP_Point_x', 'RIGHT_HIP_Point_y',
        'RIGHT_KNEE_Point_x', 'RIGHT_KNEE_Point_y'
    ]]
    Angles_df['Angle_At_Right_Hip'] = Angle_At_Right_Hip.apply(calculate_angle, axis=1)

    # Calculate angle at left hip
    Angle_At_Left_Hip = df[[
        'LEFT_SHOULDER_Point_x', 'LEFT_SHOULDER_Point_y',
        'LEFT_HIP_Point_x', 'LEFT_HIP_Point_y',
        'LEFT_KNEE_Point_x', 'LEFT_KNEE_Point_y'
    ]]
    Angles_df['Angle_At_Left_Hip'] = Angle_At_Left_Hip.apply(calculate_angle, axis=1)

    # Calculate angle at neck (example)
    Angle_At_Neck = df[[
        'LEFT_SHOULDER_Point_x', 'LEFT_SHOULDER_Point_y',
        'NOSE_Point_x', 'NOSE_Point_y',
        'RIGHT_SHOULDER_Point_x', 'RIGHT_SHOULDER_Point_y'
    ]]
    Angles_df['Angle_At_Neck'] = Angle_At_Neck.apply(calculate_angle, axis=1)

    label = 1 if positive else 0

    return Angles_df.assign(Label=label)

In [6]:
frame_count = 0
frame_numbers = []

In [7]:
dataset = pd.DataFrame()

VideoFiles = os.listdir('Videos')


for VideoFile in VideoFiles:
    label = False if VideoFile.startswith('W') else True
    print(f'File in process : {VideoFile}')
    
    angles_dataset = processVideo(f'Videos/{VideoFile}', positive=label)
    dataset = pd.concat([dataset, angles_dataset], ignore_index=True)
    print(dataset.info())

dataset.to_excel('Datasets/label.xlsx', index=False)
print(f"Data saved to Datasets/label.xlsx")

File in process : pullupr1.mp4
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Angle_At_Right_Elbow     174 non-null    float64
 1   Angle_At_Left_Elbow      174 non-null    float64
 2   Angle_At_Right_Shoulder  174 non-null    float64
 3   Angle_At_Left_Shoulder   174 non-null    float64
 4   Angle_At_Right_Knee      174 non-null    float64
 5   Angle_At_Left_Knee       174 non-null    float64
 6   Angle_At_Right_Hip       174 non-null    float64
 7   Angle_At_Left_Hip        174 non-null    float64
 8   Angle_At_Neck            174 non-null    float64
 9   Label                    174 non-null    int64  
dtypes: float64(9), int64(1)
memory usage: 13.7 KB
None
File in process : pullupr2.mp4
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 437 entries, 0 to 436
Data columns (total 10 columns):
 #   Column             